<h1>Sparse Convolutional Denoising Autoencoders for Genotype Imputation <span class="tocSkip"></span></h1>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Loading-data" data-toc-modified-id="Loading-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Loading data</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Preprocessing</a></span></li></ul></li><li><span><a href="#Method" data-toc-modified-id="Method-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Method</a></span><ul class="toc-item"><li><span><a href="#Load-model" data-toc-modified-id="Load-model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load model</a></span></li><li><span><a href="#Prediction-on-test-data" data-toc-modified-id="Prediction-on-test-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Prediction on test data</a></span></li></ul></li></ul></div>

# Introduction

This notebook demonstrates a case study of testing a SCDA model on yeast genotype dataset with 10% missing genotypes. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, UpSampling1D, Dropout
from tensorflow.keras.regularizers import l1
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import load_model

/Users/jchen67/Experiment/deeplearning/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jchen67/Experiment/deeplearning/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jchen67/Experiment/deeplearning/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jchen67/Expe

# Dataset

## Loading data

In [2]:
input_name = 'data/yeast_genotype_test.txt'
df_ori = pd.read_csv(input_name, sep='\t', index_col=0)
df_ori.shape

(877, 28220)

In [3]:
df_ori.head()

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
40_73,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
40_74,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
40_75,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
40_76,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
40_77,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2


## Preprocessing

In [4]:
# one hot encode
test_X = to_categorical(df_ori)
test_X.shape

(877, 28220, 3)

# Method

## Load model

In [5]:
# returns a compiled model
SCDA = load_model('model/SCDA.h5')

W0822 20:37:20.972470 140734891664832 deprecation.py:506] From /Users/jchen67/Experiment/deeplearning/venv/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 20:37:20.973669 140734891664832 deprecation.py:506] From /Users/jchen67/Experiment/deeplearning/venv/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 20:37:20.975331 140734891664832 deprecation.py:506] From /Users/jchen67/Experiment/deeplearning/venv/lib/python3.6/site-packages/tensorflo

## Prediction on test data

In [6]:
# hyperparameters
missing_perc = 0.1

In [7]:
test_X_missing = test_X.copy()
test_X_missing.shape

(877, 28220, 3)

In [8]:
def cal_prob(predict_missing_onehot):
    # calcaulate the probility of genotype 0, 1, 2
    predict_prob = predict_missing_onehot[:,:,1:3] / predict_missing_onehot[:,:,1:3].sum(axis=2, keepdims=True)
    return predict_prob[0]

In [9]:
avg_accuracy = []
for i in range(test_X_missing.shape[0]):
    # Generates missing genotypes
    missing_size = int(missing_perc * test_X_missing.shape[1])
    missing_index = np.random.randint(test_X_missing.shape[1],
                                      size=missing_size)
    test_X_missing[i, missing_index, :] = [1, 0, 0]

    # predict
    predict_onehot = SCDA.predict(test_X_missing[i:i + 1, :, :])
    # only care the missing position
    predict_missing_onehot = predict_onehot[0:1, missing_index, :]
    
    # calculate probability and save file.
    predict_prob = cal_prob(predict_missing_onehot)
    pd.DataFrame(predict_prob).to_csv('results/{}.csv'.format(df_ori.index[i]),
                                      header=[1, 2],
                                      index=False)

    # predict label
    predict_missing = np.argmax(predict_missing_onehot, axis=2)
    # real label
    label_missing_onehot = test_X[i:i + 1, missing_index, :]
    label_missing = np.argmax(label_missing_onehot, axis=2)
    # accuracy
    correct_prediction = np.equal(predict_missing, label_missing)
    accuracy = np.mean(correct_prediction)
    print('{}/{}, sample ID: {}, accuracy: {:.4f}'.format(
        i, test_X_missing.shape[0], df_ori.index[i], accuracy))

    avg_accuracy.append(accuracy)

0/877, sample ID: 40_73, accuracy: 0.9975
1/877, sample ID: 40_74, accuracy: 0.9975
2/877, sample ID: 40_75, accuracy: 0.9982
3/877, sample ID: 40_76, accuracy: 0.9979
4/877, sample ID: 40_77, accuracy: 0.9972
5/877, sample ID: 40_79, accuracy: 0.9968
6/877, sample ID: 40_80, accuracy: 0.9979
7/877, sample ID: 40_81, accuracy: 0.9986
8/877, sample ID: 40_82, accuracy: 0.9986
9/877, sample ID: 40_83, accuracy: 0.9979
10/877, sample ID: 40_84, accuracy: 0.9989
11/877, sample ID: 40_85, accuracy: 0.9968
12/877, sample ID: 40_86, accuracy: 0.9975
13/877, sample ID: 40_87, accuracy: 0.9989
14/877, sample ID: 40_88, accuracy: 0.9979
15/877, sample ID: 40_89, accuracy: 0.9979
16/877, sample ID: 40_90, accuracy: 0.9961
17/877, sample ID: 40_91, accuracy: 0.9982
18/877, sample ID: 40_92, accuracy: 0.9972
19/877, sample ID: 40_93, accuracy: 0.9993
20/877, sample ID: 40_94, accuracy: 0.9989
21/877, sample ID: 40_95, accuracy: 0.9979
22/877, sample ID: 40_96, accuracy: 0.9968
23/877, sample ID: 41

190/877, sample ID: 42_72, accuracy: 0.9965
191/877, sample ID: 42_73, accuracy: 0.9972
192/877, sample ID: 42_74, accuracy: 0.9986
193/877, sample ID: 42_75, accuracy: 0.9972
194/877, sample ID: 42_76, accuracy: 0.9982
195/877, sample ID: 42_77, accuracy: 0.9975
196/877, sample ID: 42_78, accuracy: 0.9986
197/877, sample ID: 42_79, accuracy: 0.9972
198/877, sample ID: 42_80, accuracy: 0.9979
199/877, sample ID: 42_81, accuracy: 0.9979
200/877, sample ID: 42_82, accuracy: 0.9975
201/877, sample ID: 42_83, accuracy: 0.9975
202/877, sample ID: 42_84, accuracy: 0.9989
203/877, sample ID: 42_85, accuracy: 0.9982
204/877, sample ID: 42_86, accuracy: 0.9965
205/877, sample ID: 42_87, accuracy: 0.9979
206/877, sample ID: 42_88, accuracy: 0.9989
207/877, sample ID: 42_89, accuracy: 0.9975
208/877, sample ID: 42_90, accuracy: 0.9986
209/877, sample ID: 42_91, accuracy: 0.9982
210/877, sample ID: 42_92, accuracy: 0.9979
211/877, sample ID: 42_93, accuracy: 0.9982
212/877, sample ID: 42_94, accur

378/877, sample ID: 44_69, accuracy: 0.9996
379/877, sample ID: 44_70, accuracy: 0.9986
380/877, sample ID: 44_71, accuracy: 0.9996
381/877, sample ID: 44_72, accuracy: 0.9982
382/877, sample ID: 44_73, accuracy: 0.9989
383/877, sample ID: 44_74, accuracy: 0.9975
384/877, sample ID: 44_75, accuracy: 0.9993
385/877, sample ID: 44_76, accuracy: 0.9972
386/877, sample ID: 44_77, accuracy: 0.9975
387/877, sample ID: 44_78, accuracy: 0.9965
388/877, sample ID: 44_79, accuracy: 0.9979
389/877, sample ID: 44_80, accuracy: 0.9996
390/877, sample ID: 44_81, accuracy: 0.9996
391/877, sample ID: 44_82, accuracy: 0.9972
392/877, sample ID: 44_83, accuracy: 0.9982
393/877, sample ID: 44_84, accuracy: 0.9972
394/877, sample ID: 44_85, accuracy: 0.9979
395/877, sample ID: 44_86, accuracy: 0.9986
396/877, sample ID: 44_87, accuracy: 0.9996
397/877, sample ID: 44_88, accuracy: 0.9972
398/877, sample ID: 44_89, accuracy: 0.9972
399/877, sample ID: 44_90, accuracy: 0.9982
400/877, sample ID: 44_91, accur

567/877, sample ID: 46_67, accuracy: 0.9979
568/877, sample ID: 46_68, accuracy: 0.9986
569/877, sample ID: 46_69, accuracy: 0.9972
570/877, sample ID: 46_70, accuracy: 0.9954
571/877, sample ID: 46_71, accuracy: 0.9982
572/877, sample ID: 46_72, accuracy: 0.9975
573/877, sample ID: 46_73, accuracy: 0.9982
574/877, sample ID: 46_74, accuracy: 0.9968
575/877, sample ID: 46_75, accuracy: 0.9989
576/877, sample ID: 46_76, accuracy: 0.9968
577/877, sample ID: 46_77, accuracy: 0.9986
578/877, sample ID: 46_78, accuracy: 0.9989
579/877, sample ID: 46_79, accuracy: 0.9982
580/877, sample ID: 46_80, accuracy: 0.9989
581/877, sample ID: 46_81, accuracy: 0.9965
582/877, sample ID: 46_82, accuracy: 0.9986
583/877, sample ID: 46_83, accuracy: 0.9660
584/877, sample ID: 46_84, accuracy: 0.9972
585/877, sample ID: 46_85, accuracy: 0.9972
586/877, sample ID: 46_86, accuracy: 0.9961
587/877, sample ID: 46_87, accuracy: 0.9979
588/877, sample ID: 46_88, accuracy: 0.9979
589/877, sample ID: 46_89, accur

755/877, sample ID: 48_65, accuracy: 0.9982
756/877, sample ID: 48_66, accuracy: 0.9986
757/877, sample ID: 48_67, accuracy: 0.9982
758/877, sample ID: 48_68, accuracy: 0.9986
759/877, sample ID: 48_69, accuracy: 0.9965
760/877, sample ID: 48_70, accuracy: 0.9975
761/877, sample ID: 48_71, accuracy: 0.9968
762/877, sample ID: 48_72, accuracy: 0.9989
763/877, sample ID: 48_73, accuracy: 0.9975
764/877, sample ID: 48_74, accuracy: 0.9979
765/877, sample ID: 48_75, accuracy: 0.9993
766/877, sample ID: 48_76, accuracy: 0.9979
767/877, sample ID: 48_77, accuracy: 0.9975
768/877, sample ID: 48_78, accuracy: 0.9975
769/877, sample ID: 48_79, accuracy: 0.9972
770/877, sample ID: 48_80, accuracy: 1.0000
771/877, sample ID: 48_81, accuracy: 0.9972
772/877, sample ID: 48_82, accuracy: 0.9989
773/877, sample ID: 48_83, accuracy: 0.9979
774/877, sample ID: 48_84, accuracy: 0.9972
775/877, sample ID: 48_85, accuracy: 0.9982
776/877, sample ID: 48_86, accuracy: 0.9986
777/877, sample ID: 48_87, accur

In [10]:
print('The average imputation accuracy' \
      'on test data with {} missing genotypes is {:.4f}: '
    .format(missing_perc, np.mean(avg_accuracy)))

The average imputation accuracyon test data with 0.1 missing genotypes is 0.9979: 
